In [8]:
import numpy as np
from scipy.constants import hbar,h,e,c
from scipy.constants import elementary_charge as elc
F0 = 2.0678e-15

In [9]:
e_eff=(11.45+1)/2## for all calculations
Mhz=(2*np.pi*1e6)## divide on it to obtain value in mhz
j=0.5e-3      # critical current density nA/nm^2

In [10]:
import pandas as pd

# Q1 related parameters

In [11]:
def cap_func(Q1,Qg1,Qg2):
    return Q1+(1/Qg1+1/Qg2)**(-1)

In [12]:
def Ec(C):#fF
    return np.round((elc)**2/(2*C)/h/1e9*1e15,1) # return in MHz

In [13]:
path=r'E:\Github2.0\QCreator\designs\ThreeQubitsPlusTwoQubits\maxwell_simulations'
path=r'C:\Python\Github\QCreator\designs\Single_qubit_design_pp_qubits\Single_qubits_6x10_reduced_to_4.5x8'
path=r'D:\NextCloud\Quantum Computing\ChipDesigns\Ivan_Gerhard_designs\Two_qubits_plus_coupler\Hamiltonian_creation'

In [82]:
data=pd.read_csv(path+'\Design_single_qubit_2.txt',sep='\t')# matrix should be done on pF

In [83]:
data

,Charge,ground,Q1_bottom,Q1_top,Resonator
Charge,4.433900e-02,-0.044332,-0.000001,-0.000005,-3.061000e-07
ground,-4.433200e-02,0.407970,-0.124430,-0.214430,-2.477400e-02
Q1_bottom,-1.464700e-06,-0.124430,0.157740,-0.030558,-2.754500e-03
Q1_top,-5.211000e-06,-0.214430,-0.030558,0.260080,-1.508600e-02
Resonator,-3.061000e-07,-0.024774,-0.002755,-0.015086,4.261500e-02


In [84]:
q_name='Q1_'
Q1_cap_ground_top=data[q_name+'top'][3]+data[q_name+'top'][2]
Q1_cap_ground_bottom=data[q_name+'bottom'][2]+data[q_name+'bottom'][3]
Q1_cap_top_bottom=-data[q_name+'bottom'][3]
Q1_cap=cap_func(Q1_cap_top_bottom,Q1_cap_ground_top,Q1_cap_ground_bottom)

In [85]:
Q1_cap_top_bottom,Q1_cap_ground_bottom,Q1_cap_ground_top

(0.030558, 0.127182, 0.22952199999999998)

In [86]:
Q1_cap

0.1123935471315152

In [87]:
Ec(Q1_cap)

172.3428952908961

In [88]:
def I(f,Ec):
    Ej=Ejf(f,Ec)
    I=Ej*2*np.pi/F0*h*1e18
    return I
def Ejf(f,Ec):
    Ej=(f+Ec)**2/(8*Ec)
    return Ej
def Eji(I):#nA
    return I*F0/(2*np.pi)/1e9/h/1e9
def IEj(Ej):
    return Ej/(F0/(Q2*np.pi)/1e9/h/1e9)
def f(Ej,Ec):
    return np.sqrt(8*Ej*Ec)-Ec
def Ec(C):#fF
    return elc**2/(2*C)/h/1e9*1e15

In [89]:
a1    = 0.220
a2    = 0.220 # Junction width in um
#Calculate Qubit frequency:
JJ_area = a1*a2*1e6 #in nm^2
print('Critical current density is: ',j*1e3, 'uA/um^2')
Ic = JJ_area*j #in nA
print('Critical current: ',Ic ,' nA')
print('Ej: ',Eji(Ic) ,' GHz')

print('junction area  in um: ',JJ_area*1e-6)

Critical current density is:  0.5 uA/um^2
Critical current:  24.2  nA
Ej:  12.019544209134573  GHz
junction area  in um:  0.0484


In [90]:
f(Eji(Ic),Ec(Q1_cap)/1e3)

3.8985125912909364

In [91]:
Eji(Ic)/(Ec(Q1_cap)/1e3)

69.74203484771964

### Resonator coupling

In [92]:
def line_qubit_coupling(Q1g,Q2g,Q1gc,Q2gc):
    return np.abs((Q1g*Q2gc-Q2g*Q1gc)/(Q1g+Q2gc+Q2g+Q1gc))

In [93]:
def cap_pretty_print(args,coeff):
    for i in args:
        print(np.round(i*coeff,2))# return in fF

In [94]:
Q1_cap_res_top=-data[q_name+'top'][-1]
Q1_cap_res_bottom=-data[q_name+'bottom'][-1]
Ccr=line_qubit_coupling(Q1_cap_ground_top,Q1_cap_ground_bottom,Q1_cap_res_top,Q1_cap_res_bottom)

In [95]:
cap_pretty_print([Q1_cap_res_top,Q1_cap_res_bottom,Ccr],1e3)

15.09
2.75
3.43


In [96]:
def qubit_res_coupling(Cc,Cq,wq,wr,Zr):# capacitances in fF, frequencies in GHz
    g= Cc*10**(-12)/2*np.sqrt((2*np.pi*wq*10**(9)*Zr/(Cq*10**(-12))))*wr*10**(9)
    return np.round(g/1e6,1)# return g/2pi in MHz

In [117]:
Zr=52.6
omega_r=5.7
omega_q=6.5
gqr=qubit_res_coupling(Ccr,Q1_cap,omega_q,omega_r,Zr)
gqr

42.8

### Dispersive shift

In [118]:
def dispersive_shift(gqr,wq,wr,Ec): # gqr in MHz,wq,wr in GHZ, EC in MHz and positive
    chi=gqr**2/((wq-wr)*10**(3))/(1+(wq-wr)*10**(3)/(-Ec))
    return chi # in MHz

In [119]:
chi=dispersive_shift(gqr,omega_q,omega_r,Ec(Q1_cap))
chi

-0.628736229824071

### Readout resonator properties

In [120]:
Q=20000
k=2*np.pi*omega_r*1e9/Q 
np.round(1/k*1e9,1), np.round(k/Mhz,2)# first value in ns

(558.4, 0.29)

In [121]:
Tp=(1/k)*((omega_q-omega_r)*10**3/gqr)**2
Tp*1e6#us

195.1046892527444

### Charge line

In [59]:
Q1_cap_charge_line_top=-data[q_name+'top'][0]
Q1_cap_charge_line_bottom=-data[q_name+'bottom'][0]
Cc_charge=line_qubit_coupling(Q1_cap_ground_top,Q1_cap_ground_bottom,Q1_cap_charge_line_top,Q1_cap_charge_line_bottom)
cap_pretty_print([Q1_cap_charge_line_top,Q1_cap_charge_line_bottom,Cc_charge],1e6)#in aF

76.6
93.6
10.5


In [60]:
Cc_charge*1e6

10.545244379730185

In [62]:
omega_q=4.2

In [63]:
Z0=50
Tcharge=(Cc_charge+Q1_cap)/(Z0*(2*np.pi*omega_q*Cc_charge)**2)*1e-6

In [64]:
Tcharge*1e6#in us

25026.113067607275

### Total relaxation= resonator+charge line

In [65]:
Tp*Tcharge/(Tp+Tcharge)*1e6

119.71086492265829

In [33]:
# f1=4.5
# f2=4.7
fmin=4.3
fmax=6.5

In [34]:
Ejf(fmin,Ec(Q1_cap)/1e3),Ejf(fmax,Ec(Q1_cap)/1e3)

(14.516838057820976, 32.31219111036426)

In [35]:
(I(fmin,Ec(Q1_cap)/1e3)+I(fmax,Ec(Q1_cap)/1e3))/2,(I(fmax,Ec(Q1_cap)/1e3)-I(fmin,Ec(Q1_cap)/1e3))/2

(47.14249085289066, 17.91447064790799)

In [36]:
(I(fmin,Ec(Q1_cap)/1e3)+I(fmax,Ec(Q1_cap)/1e3))/2/j/250,(I(fmax,Ec(Q1_cap)/1e3)-I(fmin,Ec(Q1_cap)/1e3))/2/j/100

(377.1399268231253, 358.2894129581598)

In [37]:
np.sqrt(250*377),np.sqrt(358*100)

(307.001628660175, 189.208879284245)

In [1]:
307*307/189

498.67195767195767